In [1]:
import os
import cv2
import numpy as np
import glob
import numpy as np
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

/Users/avaneeshkumar/.virtualenvs/cv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/avaneeshkumar/.virtualenvs/cv/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
'''Getting Path of the Emotion Files(with mail emotion) in the Emotion Folder 
he Emotion coded files (Emotion_labels.zip) - ONLY 327 of the 593 sequences have 
emotion sequences. This is because these are the only ones the fit the prototypic 
definition. Like the FACS files, there is only 1 Emotion file for each sequence 
which is the last frame (the peak frame). There should be only one entry and the 
number will range from 0-7 
(i.e. 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise).
N.B there is only 327 files- IF THERE IS NO FILE IT MEANS THAT THERE IS NO EMOTION LABEL 
(sorry to be explicit but this will avoid confusion)'''

'Getting Path of the Emotion Files(with mail emotion) in the Emotion Folder \nhe Emotion coded files (Emotion_labels.zip) - ONLY 327 of the 593 sequences have \nemotion sequences. This is because these are the only ones the fit the prototypic \ndefinition. Like the FACS files, there is only 1 Emotion file for each sequence \nwhich is the last frame (the peak frame). There should be only one entry and the \nnumber will range from 0-7 \n(i.e. 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise).\nN.B there is only 327 files- IF THERE IS NO FILE IT MEANS THAT THERE IS NO EMOTION LABEL \n(sorry to be explicit but this will avoid confusion)'

In [1]:
import os
sub = '.DS_Store'
path = "/Users/avaneeshkumar/Documents/CK+/Emotion"

#Finding Files from the directories
def findfiles(path):
    File = []
    for root, dirs, files in os.walk(path):
        #Exclude Directories and dodge "." hidden files. 
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        #Appending root 
        for file in files:
            File.append(os.path.join(root, file)) 
    return File       

files = findfiles(path)
files

# without .DS_Store, Files List
def noDS(files):
    Files_sequence = []
    #not selecting ".DS_Store"
    sub = '.DS_Store'
    for text in files:
        if sub not in text:
            Files_sequence.append(text)
    return Files_sequence     

FS = noDS(files)
FS

['/Users/avaneeshkumar/Documents/CK+/Emotion/S005/001/S005_001_00000011_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S010/002/S010_002_00000014_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S010/004/S010_004_00000019_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S010/006/S010_006_00000015_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S011/001/S011_001_00000016_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S011/002/S011_002_00000022_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S011/003/S011_003_00000014_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S011/004/S011_004_00000021_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S011/005/S011_005_00000020_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S011/006/S011_006_00000013_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S014/001/S014_001_00000029_emotion.txt',
 '/Users/avaneeshkumar/Documents/CK+/Emotion/S014/002/

In [4]:
#Computing labelled data

In [5]:
#make emotion list by reading test file
def emo_seq(txt_file):    
    k = open(txt_file, 'r')
    emo_tag=k.readlines()
    emo=int(float(emo_tag[0]))
    return emo

def find_label(img_path,labeled_data):
    for rows in labeled_data:
        if rows[0]==img_path:
            return rows[1]

def getimagePath(txt_path):
    marker = "Emotion"
    replacement = "cohn-kanade-images"
    mn = i.find(marker)
    replaced = i[:mn] + replacement + i[mn + len(marker):-12] + ".png"
    return replaced

def labeled_data(i):
        emo=emo_seq(i)
        img_path=getimagePath(i)
        y = (img_path,emo)
        return y

LD = []
for i in FS:
    x = labeled_data(i)
    LD.append(x)
    
print(LD)


[('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S005/001/S005_001_00000011.png', 3), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S010/002/S010_002_00000014.png', 7), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S010/004/S010_004_00000019.png', 1), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S010/006/S010_006_00000015.png', 5), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S011/001/S011_001_00000016.png', 7), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S011/002/S011_002_00000022.png', 6), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S011/003/S011_003_00000014.png', 4), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S011/004/S011_004_00000021.png', 1), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S011/005/S011_005_00000020.png', 3), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S011/006/S011_006_00000013.png', 5), ('/Users/avaneeshkumar/Documents/CK+/cohn-kanade-images/S014/001/S014

In [6]:
def train_test_val_split_idxs(total_rows, percent_test):
    """
    Get indexes for training, test, and validation rows, given a total number of rows.
    Assumes indexes are sequential integers starting at 0: eg [0,1,2,3,...N]

    Returns:
    --------
    training_idxs, test_idxs, val_idxs
        Both lists of integers
    """
    if percent_test + percent_test > 1.0:
        raise ValueError('percent_test and percent_val must sum to less than 1.0')

    row_range = range(total_rows)

    no_test_rows = int(total_rows*(percent_test))
    test_idxs = np.random.choice(row_range, size=no_test_rows, replace=False)
    # remove test indexes
    row_range = [idx for idx in row_range if idx not in test_idxs]

   # no_val_rows = int(total_rows*(percent_val))
   # val_idxs = np.random.choice(row_range, size=no_val_rows, replace=False)
    # remove validation indexes
    training_idxs = row_range

    #print 'Train-test-val split: %i training rows, %i test rows, %i validation rows' % (len(training_idxs), len(test_idxs), len(val_idxs))

    return training_idxs, test_idxs

In [7]:
def image_pre_harr(img_path):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    return faces

In [8]:
def img_desc_sift(img_path):
    face_cord=image_pre_harr(img_path)
    img = cv2.imread(img_path)
    if len(face_cord) == 0 :
        return []
    (x,y,w,h) = face_cord[0]
    img=img[y:y+h, x:x+w]
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(gray,None)
    #print 'image:',img_path,'# descriptors:',des.shape
    return des

In [9]:
def decs_train(train,labeled_data):
    des_train=[]
    Y=[]
    print 'computing descriptors:', len(train)
    for i,img_path in enumerate(train):
        print 'image number',i
        des=img_desc_sift(img_path)
        if len(des)==0:
            continue
        des_train.append(des)
        label=find_label(img_path,labeled_data)
        #print label
        Y.append(label)
    #des_stack_train=[desc for img in des_train for desc in img]
    return des_train,Y


In [10]:
def find_label(img_path,labeled_data):
    for rows in labeled_data:
        if rows[0]==img_path:
            return rows[1]
        

In [11]:
def cluster_features(img_descs,cluster_model):
    """
    Cluster the training features using the cluster_model
    and convert each set of descriptors in img_descs
    to a Visual Bag of Words histogram.

    Parameters:
    -----------
    X : list of lists of SIFT descriptors (img_descs)

    training_idxs : array/list of integers
        Indicies for the training rows in img_descs

    cluster_model : clustering model (eg KMeans from scikit-learn)
        The model used to cluster the SIFT features

    Returns:
    --------
    cluster_mode :
    cluster_model has been fit to the training set
    """
    n_clusters = cluster_model.n_clusters
    des_stack_train=[desc for img in img_descs for desc in img]
    all_train_descriptors = np.array(des_stack_train)

    if all_train_descriptors.shape[1] != 128:
        raise ValueError('Expected SIFT descriptors to have 128 features, got', all_train_descriptors.shape[1])

    print '%i descriptors before clustering' % all_train_descriptors.shape[0]

    # Cluster descriptors to get codebook
    print 'Using clustering model %s...' % repr(cluster_model)
    print 'Clustering on training set to get codebook of %i words' % n_clusters

    # train kmeans or other cluster model on those descriptors selected above
    cluster_model.fit(all_train_descriptors)
    print 'done clustering.'

    return cluster_model

In [12]:
def histogram(cluster_model,des,K):
    words=cluster_model.predict(des)
    img_bow_hist = np.array([np.bincount(words, minlength=K)])
    return img_bow_hist

In [13]:
#Splitting datainto test, train, and validation , clustreing using KMeans and MINIBatchKMeans
#computing descriptors of training data and producing corresponding labels
#

In [14]:
def cluster_split(labeled_data, K, percent_test):
    emo1=[]
    emo2=[]
    emo3=[]
    emo4=[]
    emo5=[]
    emo6=[]
    emo7=[]
    emotions=[1,2,3,4,5,6,7]
    
    print '\n total size of data:',len(labeled_data)
    # segregating database images based on emotion

    for rows in labeled_data:
        emo= str(rows[1])
        a='emo'+emo
        locals()[a].append(rows[0])
    #splitting data into test, train and validation emotion category-wise
    train=[]
    test=[]
    #validation=[]
    for emo in emotions:
        total_rows=len(locals()['emo'+str(emo)])
        training_idxs, test_idxs = train_test_val_split_idxs(total_rows,percent_test)
        train_emo=[locals()['emo'+str(emo)][i] for i in training_idxs]
        test_emo=[locals()['emo'+str(emo)][i] for i in test_idxs]
        #val_emo=[locals()['emo'+str(emo)][i] for i in val_idxs]
        for rows in train_emo:
            train.append(rows)
        for rows in test_emo:
            test.append(rows)
       # for rows in val_emo:
          #  validation.append(rows)
    print '\n size of training data:',len(train),'\n size of test data:',len(test)
    # getting descriptors of training set along with labels
    print '\n computing descriptors of training data and producing corresponding labels'
    des_train,Y_train = decs_train(train,labeled_data)
    # building cluster model
    cluster_model = cluster_features(des_train,cluster_model=MiniBatchKMeans(n_clusters=K))
    
    X_train=[]   # initialising array of input vectors for training set
    
    print '\n calculting histogram vectors of training set'
    for des in des_train:
        hist=histogram(cluster_model,des,K)
        X_train.append(hist)
    X_train=np.array(X_train)
    
    # building validation set in terms of labels and vectors
    #X_val=[]
   # print '\n computing descriptors of validation data and producing corresponding labels'
    #des_val,Y_val=decs_train(validation,labeled_data)
    
   # print '\n calculting histogram vectors of validation set'
   # for des in des_val:
     #   hist=histogram(cluster_model,des,K)
     #   X_val.append(hist)
    #X_val=np.array(X_val)
    
    # building test set in terms of labels and vectors
    X_test=[]
    print '\n computing descriptors of test data and producing corresponding labels'
    des_test,Y_test=decs_train(test,labeled_data)
    
    print '\n calculting histogram vectors of test set'
    for des in des_test:
        hist=histogram(cluster_model,des,K)
        X_test.append(hist)
    X_test=np.array(X_test)
    return X_train, Y_train, X_test, Y_test, cluster_model

In [15]:
ld = LD

In [16]:
#X_train, Y_train, X_val, Y_val, X_test, Y_test, cluster_mode = cluster_split(ld, 200, 0.1,0.2)

In [17]:
def run_svm(X_train, X_test, Y_train, Y_test, scoring,
    c_vals=[1, 5, 10], gamma_vals=[0.1, 0.01, 0.0001, 0.00001]):

    param_grid = [
    #   {'C': c_vals, 'kernel': ['linear']},
      {'C': c_vals, 'gamma': gamma_vals, 'kernel': ['rbf']},
     ]

    svc = GridSearchCV(SVC(), param_grid, n_jobs=-1, cv=10, scoring=scoring)
    svc.fit(X_train, Y_train)
    print 'train score (%s):'%scoring, svc.score(X_train, Y_train)
    test_score = svc.score(X_test, Y_test)
    print 'test score (%s):'%scoring, test_score

    print svc.best_estimator_

    return svc, test_score

In [18]:
scoring = 'recall_micro'
results = {}
K_vals = [100,150,200,350,400,450,500,530,550,600]

for K in K_vals:
    X_train, Y_train, X_test, Y_test, cluster_model = cluster_split(ld, K,0.1)
    X_train=X_train.reshape(len(X_train),K)
    X_test=X_test.reshape(len(X_test),K)
    print '\nSVM Scores: '
    svmGS, svm_score = run_svm(X_train, X_test, Y_train, Y_test, scoring)
    
    results[K] = dict(svmGS=svmGS,cluster_model=cluster_model,svm_score=svm_score)
    print '\n*** K=%i DONE ***\n' % K

print '**************************'
print '***** FINISHED ALL K *****'
print '**************************\n'


 total size of data: 327

 size of training data: 299 
 size of test data: 28

 computing descriptors of training data and producing corresponding labels
computing descriptors: 299
image number 0
image number 1
image number 2
image number 3
image number 4
image number 5
image number 6
image number 7
image number 8
image number 9
image number 10
image number 11
image number 12
image number 13
image number 14
image number 15
image number 16
image number 17
image number 18
image number 19
image number 20
image number 21
image number 22
image number 23
image number 24
image number 25
image number 26
image number 27
image number 28
image number 29
image number 30
image number 31
image number 32
image number 33
image number 34
image number 35
image number 36
image number 37
image number 38
image number 39
image number 40
image number 41
image number 42
image number 43
image number 44
image number 45
image number 46
image number 47
image number 48
image number 49
image number 50
image number

/Users/avaneeshkumar/.virtualenvs/cv/lib/python2.7/site-packages/sklearn/grid_search.py:438: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)


image number 2
image number 3
image number 4
image number 5
image number 6
image number 7
image number 8
image number 9
image number 10
image number 11
image number 12
image number 13
image number 14
image number 15
image number 16
image number 17
image number 18
image number 19
image number 20
image number 21
image number 22
image number 23
image number 24
image number 25
image number 26
image number 27
image number 28
image number 29
image number 30
image number 31
image number 32
image number 33
image number 34
image number 35
image number 36
image number 37
image number 38
image number 39
image number 40
image number 41
image number 42
image number 43
image number 44
image number 45
image number 46
image number 47
image number 48
image number 49
image number 50
image number 51
image number 52
image number 53
image number 54
image number 55
image number 56
image number 57
image number 58
image number 59
image number 60
image number 61
image number 62
image number 63
image number 64


image number 109
image number 110
image number 111
image number 112
image number 113
image number 114
image number 115
image number 116
image number 117
image number 118
image number 119
image number 120
image number 121
image number 122
image number 123
image number 124
image number 125
image number 126
image number 127
image number 128
image number 129
image number 130
image number 131
image number 132
image number 133
image number 134
image number 135
image number 136
image number 137
image number 138
image number 139
image number 140
image number 141
image number 142
image number 143
image number 144
image number 145
image number 146
image number 147
image number 148
image number 149
image number 150
image number 151
image number 152
image number 153
image number 154
image number 155
image number 156
image number 157
image number 158
image number 159
image number 160
image number 161
image number 162
image number 163
image number 164
image number 165
image number 166
image number 1

image number 210
image number 211
image number 212
image number 213
image number 214
image number 215
image number 216
image number 217
image number 218
image number 219
image number 220
image number 221
image number 222
image number 223
image number 224
image number 225
image number 226
image number 227
image number 228
image number 229
image number 230
image number 231
image number 232
image number 233
image number 234
image number 235
image number 236
image number 237
image number 238
image number 239
image number 240
image number 241
image number 242
image number 243
image number 244
image number 245
image number 246
image number 247
image number 248
image number 249
image number 250
image number 251
image number 252
image number 253
image number 254
image number 255
image number 256
image number 257
image number 258
image number 259
image number 260
image number 261
image number 262
image number 263
image number 264
image number 265
image number 266
image number 267
image number 2

...

done clustering.

 calculting histogram vectors of training set

 computing descriptors of test data and producing corresponding labels
computing descriptors: 28
image number 0
image number 1
image number 2
image number 3
image number 4
image number 5
image number 6
image number 7
image number 8
image number 9
image number 10
image number 11
image number 12
image number 13
image number 14
image number 15
image number 16
image number 17
image number 18
image number 19
image number 20
image number 21
image number 22
image number 23
image number 24
image number 25
image number 26
image number 27

 calculting histogram vectors of test set

SVM Scores: 
train score (recall_micro): 0.748322147651
test score (recall_micro): 0.714285714286
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

*** K=530 DONE ***


 total size o

/Users/avaneeshkumar/.virtualenvs/cv/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=300 should be larger than k=550. Setting it to 3*k
  init_size=init_size)


done clustering.

 calculting histogram vectors of training set

 computing descriptors of test data and producing corresponding labels
computing descriptors: 28
image number 0
image number 1
image number 2
image number 3
image number 4
image number 5
image number 6
image number 7
image number 8
image number 9
image number 10
image number 11
image number 12
image number 13
image number 14
image number 15
image number 16
image number 17
image number 18
image number 19
image number 20
image number 21
image number 22
image number 23
image number 24
image number 25
image number 26
image number 27

 calculting histogram vectors of test set

SVM Scores: 
train score (recall_micro): 0.768456375839
test score (recall_micro): 0.642857142857
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

*** K=550 DONE ***


 total size o

/Users/avaneeshkumar/.virtualenvs/cv/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=300 should be larger than k=600. Setting it to 3*k
  init_size=init_size)


done clustering.

 calculting histogram vectors of training set

 computing descriptors of test data and producing corresponding labels
computing descriptors: 28
image number 0
image number 1
image number 2
image number 3
image number 4
image number 5
image number 6
image number 7
image number 8
image number 9
image number 10
image number 11
image number 12
image number 13
image number 14
image number 15
image number 16
image number 17
image number 18
image number 19
image number 20
image number 21
image number 22
image number 23
image number 24
image number 25
image number 26
image number 27

 calculting histogram vectors of test set

SVM Scores: 
train score (recall_micro): 0.741610738255
test score (recall_micro): 0.607142857143
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

*** K=600 DONE ***

**************